### 1.Introduction
This notebook will guide you to run a all-or-nothing traffic assignment by assigning the origin-destination (OD) trips to their shortest paths. The example road network is based on the streets in North Berkeley. The network data is represented by `nodes_df` and `links_df` (`df` means the variable is a dataframe). The OD pairs (`od_df`) is also provided to you, where the origins are places in the Berkeley Hills aea and the destination is a virtual location in downtown Berkeley. You will be using a shortest path function provided to you to compute the shortest path between each OD pair. Your final task is to generate a csv file containing the number of traffic volume on each link. You should name this file `quiz3_yourname.csv` and submit it

**You don't need to change the code unless you see ###YOUR CODE/NAME HERE###**. But you can change the code if you have better ways of doing it. :)

### 2.Download data
Run the code block below to download the input data.

In [ ]:
# retrieve the sp code for shortest path calculation
!rm -rf sp && mkdir sp
!wget "https://github.com/UCB-CE170a/Fall2020/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/interface.py" -O sp/interface.py

# retrieve the road network and demand inputs
!rm -rf traffic_inputs && mkdir traffic_inputs
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_links.csv" -O traffic_inputs/berkeley_links.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_nodes.csv" -O traffic_inputs/berkeley_nodes.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/od_10pn.csv" -O traffic_inputs/od_10pn.csv

### 3.Import modules
Run the code block below to import modules and libraries that will be used in this notebook. We will use the `pandas` library to handle various tabular data (network links, nodes and OD pairs), as well as the `sp` module for shortest path computation.

In [ ]:
import pandas as pd
from sp import interface

### 4.Define functions
Run the code block below to execute function definitions. No need to change the code.

In [ ]:
# define a helper function to retrieve and format the shortest path between an origin-destination pair
def get_path(origin, destination, endnode2link_dict):
    """
    Retrieving the shortest path between a given origin-destination pair.

    Parameters:
    origin (int): origin node id
    destin (int): destination node id
    endnode2link_dict (dict): a dictionary that maps the start and end node to the link id

    Returns:
    route (list of int): the shortest path, represented by a list of ids of the links that form the path
    sp_dist (float): the shortest path distance
    """
    ### retrieve shortest path between the origin and destination nodes using Dijkstra's Algorithm
    shortest_path = g.dijkstra(origin, destination)
    ### calculate the path distance
    shortest_path_distance = shortest_path.distance(destination)

    if shortest_path_distance > 1e8:
        ### if path does not exist between the origin and destination, it will return a very high distance value
        route = []
    else:
        ### otherwise, a path is found. We will convert them into a list of links, where each element is the id of the link
        route = [endnode2link_dict[(start_node_id, end_node_id)] for (start_node_id, end_node_id) in shortest_path.route(destination)]
    shortest_path.clear()
    
    return route, shortest_path_distance

# define a function to initiate a dictionary, where the each dictionary key is the road id and the corresponding dictionary value is the traffic volume (set to zero) on that road.
def init_traffic_volume_dict(link2endnode_dict):
    """
    Initiate a dictionary containing the traffic volume on each road link.

    Parameters:
    link2endnode_dict (dict): a dictionary that maps the link id to its start and end node id.

    Returns:
    traffic_volume_dictionary (dict): a dictionary that maps the link id to its traffic volume. The traffic volume is set to zero since this is an initialization function.
    """
    traffic_volume_dictionary = {}
    for link_id in link2endnode_dict.keys():
        traffic_volume_dictionary[link_id] = 0
    return traffic_volume_dictionary

# define a function to populate the road-volume dictionary
def update_traffic_volume_dict(traffic_volume_dictionary,route):
    """
    Populate a dictionary containing the traffic volume on each road link.

    Parameters:
    traffic_volume_dictionary (dict): a dictionary that maps the link id to its traffic volume. This dictionary is updated according to the new route information.
    route (list): a list representing a particular shortest path route. Elements in the list are the ids of links along the shorrtest path.

    Returns:
    traffic_volume_dictionary (dict): a dictionary that maps the link id to its traffic volume.
    """
    for path_link in route:
        traffic_volume_dictionary[path_link] += 1
    return traffic_volume_dictionary

# function to convert road-volume dictionary to a dataframe
def convert_tv_dict2df(traffic_volume_dictionary):
    """
    Convert the dictionary of {link_id: link_volume} to a Pandas dataframe.

    Parameters:
    traffic_volume_dictionary (dict): a dictionary that maps the link id to its traffic volume.

    Returns:
    traffic_volume_df (dataframe): a dataframe consists of 2 columns and N rows, with N equal to the numbers of links in the network where traffic volume is not zero. Each row is the id and traffic volume of a link.
    """
    valid_link_volume = [(link,volume) for link, volume in traffic_volume_dictionary.items() if volume > 0]
    traffic_volume_df = pd.DataFrame(valid_link_volume, columns=['link_id', 'volume'])
    return traffic_volume_df

### 5.Read in initial data


##### 5.a Read in network nodes and display the first two rows of data

In [ ]:
# read network nodes
nodes_df = pd.read_csv('traffic_inputs/berkeley_nodes.csv')
display(nodes_df.head(2))

##### 5.b Read in network links and display the first two rows of data

In [ ]:
# read network links
links_df = pd.read_csv('traffic_inputs/berkeley_links.csv')
display(links_df.head(2))

##### 5.c Read the OD-pairs and display the first two rows of data

In [ ]:
# read travel demand (OD-pairs)
od_df = pd.read_csv('traffic_inputs/od_10pn.csv')
display(od_df.head(2))

### 6.Construct a graph object and lookup dictionaries based on the network input data


In [ ]:
# make road network graph, which we need to run the shortest path calculation
# the `interface.from_dataframe()` in the `sp` module constructs a graph based on a dataframe of network links
# the arguments to `interface.from_dataframe()` are: name of the network links dataframe, the column that contains the start node id of the link, the column that contains the end node id of the link and the weight column
# we use `fft` as the weight, which is short for `free flow travel time`
# for more information on the `sp` module, please refer to https://github.com/cb-cities/sp/tree/dataframe
g = interface.from_dataframe(links_df, 'start_node_id', 'end_node_id', 'fft')

In [ ]:
# create dictionaries between node_id and link_id for quick lookup
endnode2link_dict = {(getattr(link, 'start_node_id'), getattr(link, 'end_node_id')): getattr(link, 'link_id') for link in links_df.itertuples()}
link2endnode_dict = {getattr(link, 'link_id'): (getattr(link, 'start_node_id'), getattr(link, 'end_node_id')) for link in links_df.itertuples()}

### 7.Path calculation example
Below is an example of computing the shortest path for one od pair, e.g., the path from node 91 to node 237.

- The `get_path()` function takes the input returns: (1) the shortest path between each od pair as a list, and (2) the shortest path distance. In the first return value, each list element denotes a road link by its id. For instance, in the example here, the path from node 91 and node 237 is `[747, 266, 198, 39, 41, 34, 110, 189, 18, 219, 827]`. Its first element `747` denotes a road link that connects node 91 and node 155.
- The `traffic_volume_dictionary` is a dictionary-type variable that we use to count the traffic volume on each link. This dictionary's keys are road ids and values are number of trips (i.e., traffic volumes) that use each link. Since the dictionary is initiated to be empty, we use a `try ... except ...` block to update it. If `try ...` fails, it means that the particular road link id has not been added to the dictionary yet, so we will move on to the `except ...` clause and create a new key in the dictionary corresponding to the link id.

In [ ]:
### example of path calculation
origin = 91      ### the origin node id of a trip
destination = 237     ### the end node id of a trip
route, distance = get_path(origin, destination, endnode2link_dict) ### get the shortest path between the origin and destination of the trip
print('The shortest path between {} and {} is {}'.format(origin, destination, route))
print('The path/route object is a {}, the first elment is {}'.format(type(route), route[0]))
print('The trip travel time is {:.2f} minutes.\n'.format(distance/60))

### example of traffic volume calculation
traffic_volume_dictionary = init_traffic_volume_dict(link2endnode_dict)
traffic_volume_dictionary = update_traffic_volume_dict(traffic_volume_dictionary, route)

print( 'Link {}, connecting node {} and {}, has {} trip(s) passed through it (so far)'.format( 
    747, link2endnode_dict[747][0], link2endnode_dict[747][1], traffic_volume_dictionary[747] ) )

### example of converting dictionary to dataframe that contains traffic volume for each road link
traffic_volume_df = convert_tv_dict2df(traffic_volume_dictionary)
traffic_volume_df.head()

### 8.Your tasks: compute shortest paths between all OD pairs and get traffic volume
Given the inputs (`links_df`, `nodes_df`, `od_df`, `g`, and the `get_path` fuction),
- Task 1: calculate the path for each od pair
- Task 2: compute the number of trips that pass each link (i.e., traffic volume on this link)
- Task 3: save the result of Task 2 in a CSV file `quiz3_[your_name].csv`. The file should look like:

| link_id | volume      |
|---------|-------------|
| 0       | ...         |
| 1       | ...         |
| ...     | ...         |

Hints:
- Task 1: Loop through each `od` pair and use the `get_path()` function in the code block above to compute the path for each od pair. The `get_path()` function is reusable, so you just need to call it and no need to define it every time you compute the path.
- Task 2: Initiate an empty `traffic_volume_dictionary` outside of the loop that iterates through each od pair. The key of the dictionary should be link_id (e.g., `747`), and the value corresponding to each key denotes the traffic volume on this link. For each shortest path result, update the `traffic_volume_dictionary` using the `update_traffic_volume_dict()` function.
- Beware that links are directed, meaning that link with end nodes `(91, 155)` is different from link whose end nodes are `(155, 91)`.
- **The method in the hints above are not the most efficient. It is just one way to complete the tasks. If you are comfortable of other ways, e.g., using Pandas groupby and merge functions, please choose a method that works the best for you.**

In [ ]:
# Task 1
### iterate through each OD-pair, compute the shortest path and append the route to collections of all routes (the `routes` variable)
origins = od_df['origin_node_id']
destinations = od_df['destin_node_id']

routes = []
for origin, destination in zip(origins,destinations):
  ### YOUR CODE HERE ###

In [ ]:
# Task 2

### initate dictionary holding the traffic volume for each road link
traffic_volume_dictionary = ### YOUR CODE HERE ###

### update the dictionary with each shortest path computed in the task above
for route in routes:
  ### YOUR CODE HERE ###


In [ ]:
# Task 3
### create a dataframe that counts the traffic volume on each road link under the given travel demand
### name the output file as "quiz3_[your_name].csv"

traffic_volume_df =  ### YOUR CODE HERE ###
your_name = 'tester'        ### YOUR NAME HERE avoid space ### 
traffic_volume_df.to_csv('quiz3_{}.csv'.format(your_name), index=False)